In [20]:
# import libraries
import numpy as np
import os
import sklearn
import sklearn.neighbors
import matplotlib.pyplot as plt
from IPython import display
%matplotlib inline
import itertools
import time
import random
import load_data

import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import backend as K
K.set_image_dim_ordering('tf')

In [2]:
data_dir = './input'# define here the directory where you have your data

In [3]:
# labels for the three classes in this classification problem.
# These are the same names used in folders for raw training data
cervixTypes = ["Type_1", "Type_2", "Type_3"]
n_classes = len(cervixTypes)
print('Classification problem with {} classes'.format(n_classes))

Classification problem with 3 classes


In [4]:
data, labels = load_data.load_train()

load from hdf5 file

 Doooone :)


In [5]:
samples_per_class = [249, 781, 450]

In [6]:
def split_training_validation_datasets(x, y, samples_per_class, val_percentage=0.3, val_balanced=True):
    """
    Derive a training and a validation datasets from a given dataset with
    data (x) and labels (y). By default, the validation set is 30% of the
    training set, and it has balanced samples across classes. When balancing,
    it takes the 30% of the class with less samples as reference.
    """
    n_cv_samples_per_class = int(np.amin(samples_per_class) * 0.3)
    
    mask = np.zeros(x.shape[0], dtype=bool)
    low = 0
    high = 0
     
    for c in samples_per_class:
        high = high + c
        ids_cv = np.random.choice(np.arange(low,high),n_cv_samples_per_class, replace = False)
        mask[ids_cv] = True
        low = low + c   
    
    x_train = x[~mask,:, :, :]
    y_train = y[~mask, :]

    x_validation = x[mask,:, :, :]
    y_validation = y[mask, :]
    
    return x_train, y_train, x_validation, y_validation

In [7]:
x_train, y_train, x_validation, y_validation = split_training_validation_datasets(data, labels, samples_per_class)

In [2]:
batch_size = 8
num_classes = 3
epochs = 50

In [9]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
train_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size)

# this is a similar generator, for validation data
validation_generator = test_datagen.flow(x_validation, y_validation, batch_size=batch_size)

In [3]:
img_rows, img_cols = 400, 400
input_shape = (img_rows, img_cols, 3)

In [10]:
datagen = ImageDataGenerator()

    # build the VGG16 network
model = applications.VGG16(include_top=False, weights='imagenet')

bottleneck_features_train = model.predict(x_train, batch_size= batch_size, verbose=1)
np.save(open('bottleneck_features_train.npy', 'w'), bottleneck_features_train)

1258/1258 [==============================] - 3787s  


TypeError: predict_generator() got an unexpected keyword argument 'batch_size'

In [11]:
bottleneck_features_validation = model.predict(x_validation, batch_size= batch_size, verbose=1)
np.save(open('bottleneck_features_validation.npy', 'w'), bottleneck_features_validation)

222/222 [==============================] - 650s    


In [20]:
np.save(open('x_train.npy', 'w'), x_train)
np.save(open('y_train.npy', 'w'), y_train)
np.save(open('x_validation.npy', 'w'), x_validation)
np.save(open('y_validation.npy', 'w'), y_validation)

In [7]:
top_model_weights_path = 'bottleneck_fc_model.h5'

train_data = np.load(open('bottleneck_features_train.npy'))
validation_data = np.load(open('bottleneck_features_validation.npy'))
y_train = np.load(open('y_train.npy'))
y_validation = np.load(open('y_validation.npy'))

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer=keras.optimizers.Adadelta(),
                  loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

model.fit(train_data, y_train,
              epochs=epochs,
              batch_size=batch_size,
              verbose=1,
              validation_data=(validation_data, y_validation))
model.save_weights(top_model_weights_path)

Train on 1258 samples, validate on 222 samples
Epoch 1/50
1258/1258 [==============================] - 83s - loss: 1.9122 - acc: 0.5024 - val_loss: 1.1920 - val_acc: 0.3333
Epoch 2/50
1258/1258 [==============================] - 81s - loss: 1.0241 - acc: 0.5437 - val_loss: 1.1457 - val_acc: 0.3333
Epoch 3/50
1258/1258 [==============================] - 82s - loss: 0.9984 - acc: 0.5564 - val_loss: 1.1006 - val_acc: 0.3333
Epoch 4/50
1258/1258 [==============================] - 81s - loss: 0.9988 - acc: 0.5485 - val_loss: 1.1846 - val_acc: 0.3333
Epoch 5/50
1258/1258 [==============================] - 81s - loss: 0.9792 - acc: 0.5564 - val_loss: 1.2898 - val_acc: 0.3378
Epoch 6/50
1258/1258 [==============================] - 82s - loss: 0.9550 - acc: 0.5723 - val_loss: 1.2049 - val_acc: 0.3333
Epoch 7/50
1258/1258 [==============================] - 83s - loss: 0.9460 - acc: 0.5660 - val_loss: 1.2876 - val_acc: 0.3333
Epoch 8/50
1258/1258 [==============================] - 81s - loss: 0.9

In [41]:
x_train = np.load(open('x_train.npy'))
x_validation = np.load(open('x_validation.npy'))

In [42]:
from keras.layers import Input
from keras.models import Model
weights_path = './vgg16_weights.h5'

# build the VGG16 network
input_tensor = Input(shape=(400,400,3))
base_model = applications.VGG16(weights='imagenet',include_top= False,input_tensor=input_tensor)
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(3, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
model = Model(input= base_model.input, output= top_model(base_model.output))

# # build a classifier model to put on top of the convolutional model
# x = base_model.output
# x = Flatten()(x)
# x = Dense(256, activation='relu') (x)
# x = Dropout(0.5) (x)
# predictions = Dense(3, activation='softmax')(x)

# # note that it is necessary to start with a fully-trained
# # classifier, including the top classifier,
# # in order to successfully do fine-tuning
# x.load_weights(top_model_weights_path)

# # add the model on top of the convolutional base
# model = Model(input=base_model.input, output=predictions)

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size)

# this is a similar generator, for validation data
validation_generator = test_datagen.flow(x_validation, y_validation, batch_size=batch_size)

# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=1258,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=222)

Model loaded.


/Users/riccardosamperna/Programming/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:23: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("se..., inputs=Tensor("in...)`
/Users/riccardosamperna/Programming/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:71: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=157, epochs=50, validation_steps=222)`


Epoch 1/50
157/157 [==============================] - 8944s - loss: 1.3303 - acc: 0.3583 - val_loss: 1.7568 - val_acc: 0.3337
Epoch 2/50
157/157 [==============================] - 9087s - loss: 1.3162 - acc: 0.3941 - val_loss: 1.7545 - val_acc: 0.3341
Epoch 3/50
157/157 [==============================] - 9017s - loss: 1.3000 - acc: 0.3718 - val_loss: 1.7609 - val_acc: 0.3347
Epoch 4/50
156/157 [============================>.] - ETA: 24s - loss: 1.3506 - acc: 0.3886

KeyboardInterrupt: 